In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import random

Tokenizers

In [2]:
class CharTokenizer:
    def __init__(self, text):
        self.text = text
        self.vocab = sorted(list(set(text)))
        self.vocab_size = len(self.vocab)
        self.char_to_int = { ch: i for i, ch in enumerate(self.vocab) }
        self.int_to_char = { i: ch for i, ch in enumerate(self.vocab) }

    def encode(self, s):
        return [self.char_to_int[ch] for ch in s]

    def decode(self, ints):
        return ''.join(self.int_to_char[i] for i in ints)

class TikTokenizer:
    def __init__(self):
        import tiktoken
        self.enc = tiktoken.get_encoding('gpt2')
        self.vocab = [self.enc.decode([i]) for i in range(self.enc.n_vocab)]
        self.vocab_size = self.enc.n_vocab

    def encode(self, s):
        return self.enc.encode(s)

    def decode(self, ints):
        return self.enc.decode(ints)

Splitting Methods

In [3]:
class BasicSplit:
    def __init__(self, val_fraction=0.1):
        self.val_fraction = val_fraction

    def split(self, data):
        n = int(self.val_fraction * len(data))
        train_data = data[:(1-n)]
        val_data   = data[(1-n):]
        return train_data, val_data

class ChunkSplit:
    def __init__(self, val_fraction=0.1, num_chunks=12, encode=lambda x: x):

        self.val_fraction = val_fraction
        self.num_chunks   = num_chunks
        self.encode       = encode

    def split(self, data):
        newline_token = self.encode("\n")

        val_fraction = self.val_fraction
        num_chunks   = self.num_chunks

        total_len = data.size(0)
        val_len   = int(val_fraction * total_len)
        chunk_len = val_len // num_chunks
        remainder = val_len % num_chunks

        # split the entire dataset into chunks
        chunks = []
        idx = 0
        while idx + chunk_len <= total_len:
            chunks.append(data[idx : idx + chunk_len])
            idx += chunk_len

        # the leftover chunk (if any) after slicing out as many `chunk_len` blocks as possible. We always put this leftover into training by default
        leftover = data[idx:]

        val_indices = sorted(random.sample(range(len(chunks)), num_chunks))

        # build the validation data (insert newline_token only between non-consecutive chunks)
        val_data = []
        prev_idx = -2  # something not adjacent to first pick 
        for i in val_indices:
            # if the current chunk is NOT directly after the previous one and val_data already has content,
            # then insert a newline token
            if i != prev_idx + 1 and len(val_data) > 0:
                val_data.append(torch.tensor(newline_token, dtype=torch.int64))
            val_data.append(chunks[i])
            prev_idx = i

        # concatenate chosen val chunks
        val_data = torch.cat(val_data) if val_data else torch.empty(0, dtype=torch.int64)

        # if we have a remainder that is significant, tack data from train onto the end. This is ugly and bad and should be changed
        # this ensures total validation tokens == val_len but can also result in a small amount of overlap (up to chunk_length -1 tokens) between val and train
        if remainder > 100:
            val_data = torch.cat((val_data, data[-remainder:]))

        # everything else is training: the unchosen chunks plus leftover
        train_indices = sorted(set(range(len(chunks))) - set(val_indices))
        train_data_list = []
        prev_idx = -2  # so first chunk won't auto-insert a newline
        for i in train_indices:
            # if this chunk is not consecutive to the previous one, insert a newline
            if i != prev_idx + 1 and len(train_data_list) > 0:
                train_data_list.append(torch.tensor(newline_token, dtype=torch.int64))
            train_data_list.append(chunks[i])
            prev_idx = i

        train_data_list.append(torch.tensor(newline_token, dtype=torch.int64))
        train_data_list.append(leftover)
        train_data = torch.cat(train_data_list) if train_data_list else torch.empty(0, dtype=torch.int64)
        return train_data, val_data


Load Data

In [4]:
input_file_path = "datasets/tinyshakespeare.txt"
# input_file_path = "datasets/big.txt"
with open(input_file_path, 'r', encoding='utf-8') as f:
    train_text = f.read()

Hyper Params

In [5]:
tokenizer = CharTokenizer(train_text)
encode = tokenizer.encode
decode = tokenizer.decode
vocab = tokenizer.vocab
vocab_size = tokenizer.vocab_size
val_fraction = 0.1
splitter = BasicSplit(val_fraction)

In [6]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
dropout = 0.0

Make Training Data

In [7]:
data = torch.tensor(encode(train_text), dtype=torch.long)
train_data, val_data = splitter.split(data)

Data Loading

In [8]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        # lower-triangular mask for future tokens (causal attention)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        # project x to key, query, value
        k = self.key(x)    # (B, T, head_size)
        q = self.query(x)  # (B, T, head_size)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, T)
        # mask out future positions for causal language modeling
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        # softmax to get attention weights
        wei = F.softmax(wei, dim=-1)  # (B, T, T)

        # weighted aggregation of the values
        v = self.value(x)  # (B, T, head_size)
        out = wei @ v       # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
        return out

In [11]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

Bigram Model

In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.postion_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_heads = MultiHeadAttention(4, n_embd//4)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B, T, C) or (B, T, n_embd)
        pos_emb = self.postion_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.sa_heads(x) # apply a head of self attention
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [13]:
model = BigramLanguageModel()
model = model.to(device)

Training

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Generate From the Model

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))